## Need: Dataset, Column Report, Row Report

In [20]:
import pandas as pd

# Load Excel file with column lists
allCols_df = pd.read_excel(
    '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/Group1Data_ColumnReport.xlsx'
)

indicator_to_source = dict(zip(allCols_df["Indicator"], allCols_df["Source Table"]))

LifeExpCols_df = pd.read_excel(
    '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/Group1Data_ColumnReport.xlsx',
    sheet_name='Life Expectancy Columns'
)
UnemCols_df = pd.read_excel(
    '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/Group1Data_ColumnReport.xlsx',
    sheet_name='Unemployment Columns'
)

# Load main dataset
dataset_df = pd.read_csv(
    '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/Group1Dataset_LCU_removed(except_exchange_rate).csv'
)

# ---------------------------------------------------------
# 1. Extract column lists from the Excel sheets
# ---------------------------------------------------------
life_exp_cols = LifeExpCols_df['Indicator'].tolist()
unem_cols = UnemCols_df['Indicator'].tolist()

# ---------------------------------------------------------
# 2. Retain only the columns that exist in the dataset
# ---------------------------------------------------------
valid_life_exp_cols = [col for col in life_exp_cols if col in dataset_df.columns]
valid_unem_cols = [col for col in unem_cols if col in dataset_df.columns]

# ---------------------------------------------------------
# 3. Define a helper function to ensure essential columns appear at the beginning
# ---------------------------------------------------------
def ensure_essential_columns_at_start(cols, df, essential_cols):
    """
    Ensures that the essential columns (if they exist in the dataset)
    are placed at the beginning of the columns list.
    """
    # Only consider essential columns that exist in the dataset
    valid_essentials = [col for col in essential_cols if col in df.columns]
    # Remove any occurrence of the essential columns from the existing list
    remaining = [col for col in cols if col not in valid_essentials]
    # Return the new list with essential columns first
    return valid_essentials + remaining

# Define essential columns in the desired order
essential_cols = ['CountryShortName', 'Year']

# ---------------------------------------------------------
# 4. Update the column lists for each group so that essential columns are first
# ---------------------------------------------------------
final_life_exp_cols = ensure_essential_columns_at_start(valid_life_exp_cols, dataset_df, essential_cols)
final_unem_cols = ensure_essential_columns_at_start(valid_unem_cols, dataset_df, essential_cols)

# ---------------------------------------------------------
# 5. Filter the dataset using the final column lists
# ---------------------------------------------------------
life_exp_data = dataset_df[final_life_exp_cols]
unem_data = dataset_df[final_unem_cols]

In [21]:
# ---------------------------
# Data Loading Functions
# ---------------------------
def load_dataset(dataset_path):
    """Load a CSV dataset."""
    return pd.read_csv(dataset_path)

def load_excel_sheet(excel_path, sheet_name=None):
    """Load an Excel sheet (or the first sheet if sheet_name is None)."""
    return pd.read_excel(excel_path, sheet_name=sheet_name)

# ---------------------------
# Filtering Functions
# ---------------------------
def get_valid_columns(df, columns):
    """Return the subset of columns that exist in the DataFrame."""
    return [col for col in columns if col in df.columns]

def ensure_essential_columns_at_start(cols, df, essential_cols):
    """
    Ensure that essential columns (if they exist in df)
    are placed at the beginning of the list.
    """
    valid_essentials = [col for col in essential_cols if col in df.columns]
    remaining = [col for col in cols if col not in valid_essentials]
    return valid_essentials + remaining

def filter_dataset(df, columns, essential_cols=['CountryShortName', 'Year']):
    """
    Filter the DataFrame to only include the valid columns from the provided list.
    Essential columns will be moved to the beginning.
    """
    valid_cols = get_valid_columns(df, columns)
    final_cols = ensure_essential_columns_at_start(valid_cols, df, essential_cols)
    return df[final_cols]

# ---------------------------
# Missing Data Report Functions
# ---------------------------
def create_missing_report_by_column_with_source(df, mapping):
    """
    Create a DataFrame reporting the count and percentage of missing values for each column,
    and add a column for the Source Table based on the mapping.
    """
    # Create missing values report with column names as a column named "Column"
    report = pd.DataFrame({
        'missing_count': df.isna().sum(),
        'missing_percent': df.isna().mean() * 100
    }).reset_index().rename(columns={'index': 'Column'})
    
    # Add the Source Table column by mapping the "Column" names to the dictionary
    report["Source Table"] = report["Column"].map(mapping)
    return report

# ---------------------------
# Utility Functions
# ---------------------------
def save_csv(df, output_path, index=False):
    """Save a DataFrame to a CSV file."""
    df.to_csv(output_path, index=index)

In [22]:
# ---------------------------
# Main Workflow
# ---------------------------
# Define file paths
excel_path = '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/Group1Data_ColumnReport.xlsx'
dataset_path = '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/Group1Dataset_LCU_removed(except_exchange_rate).csv'

# Load Excel sheets containing the column lists
allCols_df = load_excel_sheet(excel_path)
life_exp_cols_df = load_excel_sheet(excel_path, sheet_name='Life Expectancy Columns')
unem_cols_df = load_excel_sheet(excel_path, sheet_name='Unemployment Columns')

# Extract column lists from the Excel sheets
life_exp_cols = life_exp_cols_df['Indicator'].tolist()
unem_cols = unem_cols_df['Indicator'].tolist()

# Load the main dataset
dataset_df = load_dataset(dataset_path)

# Filter datasets using the lists (ensuring CountryShortName and Year are first)
life_exp_data = filter_dataset(dataset_df, life_exp_cols)
unem_data = filter_dataset(dataset_df, unem_cols)

# Generate missing reports for Life Expectancy data
life_exp_missing_report = create_missing_report_by_column_with_source(life_exp_data, indicator_to_source)

# Generate missing reports for Unemployment data
unem_missing_report = create_missing_report_by_column_with_source(unem_data, indicator_to_source)

# Save the filtered datasets to CSV files
save_csv(life_exp_data, '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/LifeExpectancyDataset.csv')
save_csv(unem_data, '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/UnemploymentDataset.csv')

# Save the missing reports for Life Expectancy data
save_csv(life_exp_missing_report, '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/LifeExp_MissingReport.csv')

# Save the missing reports for Unemployment data
save_csv(unem_missing_report, '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/Unem_MissingReport.csv')

In [25]:
columns_to_keep = [ "CountryShortName", "Year",
    "GOAL 13: Climate Action (5 year moving average)", "Pillar 3 - Data Products - Score", "GOAL 10: Reduced Inequality (5 year moving average)", "GOAL 11: Sustainable Cities and Communities (5 year moving average)", "GOAL 12: Responsible Consumption and Production (5 year moving average)", "GOAL 15: Life on Land (5 year moving average)", "GOAL 16: Peace and Justice Strong Institutions (5 year moving average)", "GOAL 17: Partnerships to achieve the Goal (5 year moving average)", "GOAL 1: No Poverty (5 year moving average)", "Health/Demographic survey (Availability score over 10 years)", "Labor Force Survey (Availability score over 10 years)", "General government final consumption expenditure (constant 2015 US$)", "General government final consumption expenditure (annual % growth)", "Final consumption expenditure (constant 2015 US$)", "Final consumption expenditure (annual % growth)", "Exports of goods and services (% of GDP)", "External balance on goods and services (% of GDP)", "Communications, computer, etc. (% of service imports, BoP)", "Communications, computer, etc. (% of service exports, BoP)", "Exports of goods, services and primary income (BoP, current US$)", "Adjusted net national income (current US$)", "Adjusted net national income per capita (current US$)", "Agriculture, forestry, and fishing, value added per worker (constant 2015 US$)", "Adjusted savings: education expenditure (% of GNI)", "GDP per capita, PPP (constant 2021 international $)", "GDP, PPP (constant 2021 international $)", "GNI per capita, PPP (current international $)", "GNI, PPP (current international $)", "Adjusted savings: energy depletion (% of GNI)", "Agriculture, forestry, and fishing, value added (% of GDP)", "GDP per capita, PPP (current international $)", "GDP, PPP (current international $)", "GDP (constant 2015 US$)", "GDP per capita (constant 2015 US$)", "GNI per capita, Atlas method (current US$)", "GNI, Atlas method (current US$)", "Adjusted savings: carbon dioxide damage (% of GNI)", "GNI (current US$)", "GDP growth (annual %)", "GDP per capita growth (annual %)", "GDP (current US$)", "GDP per capita (current US$)", "Preprimary education, duration (years)", "School enrollment, primary, female (% gross)", "School enrollment, primary, male (% gross)", "School enrollment, primary (% gross)", "Primary education, pupils", "Compulsory education, duration (years)", "Broad money (% of GDP)", "Automated teller machines (ATMs) (per 100,000 adults)", "Domestic credit to private sector (% of GDP)", "Commercial bank branches (per 100,000 adults)", "Claims on central government, etc. (% GDP)", "Monetary Sector credit to private sector (% GDP)", "Domestic credit to private sector by banks (% of GDP)", "Official exchange rate (LCU per US$, period average)", "Inflation, GDP deflator (annual %)", "Inflation, GDP deflator: linked series (annual %)", "Proportion of seats held by women in national parliaments (%)", "Women Business and the Law Index Score (scale 1-100)", "Insurance and financial services (% of commercial service exports)", "Ores and metals imports (% of merchandise imports)", "Fuel imports (% of merchandise imports)", "International tourism, expenditures for travel items (current US$)", "International tourism, receipts for travel items (current US$)", "Transport services (% of commercial service exports)", "Insurance and financial services (% of commercial service imports)", "Travel services (% of commercial service exports)", "Transport services (% of commercial service imports)", "Travel services (% of commercial service imports)", "Computer, communications and other services (% of commercial service exports)", "Computer, communications and other services (% of commercial service imports)", "Export unit value index (2015 = 100)", "Import unit value index (2015 = 100)", "Refugee population by country or territory of asylum", "Share of youth not in education, employment or training, female (% of female youth population) (modeled ILO estimate)", "Share of youth not in education, employment or training, male (% of male youth population) (modeled ILO estimate)", "Share of youth not in education, employment or training, total (% of youth population) (modeled ILO estimate)", "Refugee population by country or territory of origin", "GDP per person employed (constant 2021 PPP $)", "Fuel exports (% of merchandise exports)", "International tourism, number of arrivals", "Employment to population ratio, 15+, female (%) (modeled ILO estimate)", "Employment to population ratio, 15+, male (%) (modeled ILO estimate)", "Employment to population ratio, 15+, total (%) (modeled ILO estimate)", "Employment to population ratio, ages 15-24, female (%) (modeled ILO estimate)", "Employment to population ratio, ages 15-24, male (%) (modeled ILO estimate)", "Employment to population ratio, ages 15-24, total (%) (modeled ILO estimate)", "Labor force participation rate for ages 15-24, female (%) (modeled ILO estimate)", "Labor force participation rate for ages 15-24, male (%) (modeled ILO estimate)", "Labor force participation rate for ages 15-24, total (%) (modeled ILO estimate)", "Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)", "Labor force participation rate, female (% of female population ages 15-64) (modeled ILO estimate)", "Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate)", "Labor force participation rate, male (% of male population ages 15-64) (modeled ILO estimate)", "Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)", "Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)", "Unemployment, youth female (% of female labor force ages 15-24) (modeled ILO estimate)", "Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)", "Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)", "Fluorinated greenhouse gases (F-gases) emissions from Industrial Processes (Mt CO2e)", "Total greenhouse gas emissions including LULUCF (Mt CO2e)", "Carbon dioxide (CO2) net fluxes from LULUCF - Deforestation (Mt CO2e)", "Carbon dioxide (CO2) net fluxes from LULUCF - Forest Land (Mt CO2e)", "Carbon dioxide (CO2) net fluxes from LULUCF - Total excluding non-tropical fires (Mt CO2e)", "Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)", "PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)", "Aquaculture production (metric tons)", "Total fisheries production (metric tons)", "Cereal production (metric tons)", "Land under cereal production (hectares)", "Access to electricity, rural (% of rural population)", "Fertilizer consumption (kilograms per hectare of arable land)", "Carbon dioxide (CO2) emissions from Fugitive Emissions (Energy) (Mt CO2e)", "Nitrous oxide (N2O) emissions from Fugitive Emissions (Energy) (Mt CO2e)", "Renewable internal freshwater resources per capita (cubic meters)", "Renewable internal freshwater resources, total (billion cubic meters)", "Access to clean fuels and technologies for cooking, rural (% of rural population)", "Access to clean fuels and technologies for cooking, urban (% of urban population)", "Carbon intensity of GDP (kg CO2e per 2021 PPP $)", "Carbon intensity of GDP (kg CO2e per constant 2021 US$ of GDP)", "Rural population growth (annual %)", "Natural gas rents (% of GDP)", "Permanent cropland (% of land area)", "Carbon dioxide (CO2) emissions (total) excluding LULUCF (% change from 1990)", "Carbon dioxide (CO2) emissions (total) excluding LULUCF (Mt CO2e)", "Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)", "Carbon dioxide (CO2) emissions from Building (Energy) (Mt CO2e)", "Carbon dioxide (CO2) emissions from Industrial Combustion (Energy) (Mt CO2e)", "Carbon dioxide (CO2) emissions from Industrial Processes (Mt CO2e)", "Carbon dioxide (CO2) emissions from Power Industry (Energy) (Mt CO2e)", "Carbon dioxide (CO2) emissions from Transport (Energy) (Mt CO2e)", "Methane (CH4) emissions (total) excluding LULUCF (% change from 1990)", "Methane (CH4) emissions (total) excluding LULUCF (Mt CO2e)", "Methane (CH4) emissions from Agriculture (Mt CO2e)", "Methane (CH4) emissions from Building (Energy) (Mt CO2e)", "Methane (CH4) emissions from Fugitive Emissions (Energy) (Mt CO2e)", "Methane (CH4) emissions from Industrial Combustion (Energy) (Mt CO2e)", "Methane (CH4) emissions from Power Industry (Energy) (Mt CO2e)", "Methane (CH4) emissions from Transport (Energy) (Mt CO2e)", "Methane (CH4) emissions from Waste (Mt CO2e)", "Nitrous oxide (N2O) emissions (total) excluding LULUCF (% change from 1990)", "Nitrous oxide (N2O) emissions (total) excluding LULUCF (Mt CO2e)", "Nitrous oxide (N2O) emissions from Agriculture (Mt CO2e)", "Nitrous oxide (N2O) emissions from Building (Energy) (Mt CO2e)", "Nitrous oxide (N2O) emissions from Industrial Combustion (Energy) (Mt CO2e)", "Nitrous oxide (N2O) emissions from Industrial Processes (Mt CO2e)", "Nitrous oxide (N2O) emissions from Power Industry (Energy) (Mt CO2e)", "Nitrous oxide (N2O) emissions from Transport (Energy) (Mt CO2e)", "Nitrous oxide (N2O) emissions from Waste (Mt CO2e)", "Total greenhouse gas emissions excluding LULUCF (% change from 1990)", "Total greenhouse gas emissions excluding LULUCF (Mt CO2e)", "Total greenhouse gas emissions per capita excluding LULUCF (t CO2e/capita)", "Oil rents (% of GDP)", "Coal rents (% of GDP)", "Total natural resources rents (% of GDP)", "Agricultural land (% of land area)", "Arable land (% of land area)", "Crop production index (2014-2016 = 100)", "Food production index (2014-2016 = 100)", "Forest area (% of land area)", "Forest area (sq. km)", "Land area (sq. km)", "Livestock production index (2014-2016 = 100)", "Population density (people per sq. km of land area)", "Energy intensity level of primary energy (MJ/$2017 PPP GDP)", "Access to electricity (% of population)", "Access to electricity, urban (% of urban population)", "Arable land (hectares per person)", "Capture fisheries production (metric tons)", "Forest rents (% of GDP)", "Mineral rents (% of GDP)", "Renewable energy consumption (% of total final energy consumption)", "Rural population", "Rural population (% of total population)", "Surface area (sq. km)", "Urban population", "Urban population (% of total population)", "Urban population growth (annual %)", "Prevalence of stunting, height for age (modeled estimate, % of children under 5)", "Prevalence of overweight (modeled estimate, % of children under 5)", "Prevalence of anemia among children (% of children ages 6-59 months)", "Prevalence of anemia among non-pregnant women (% of women ages 15-49)", "Prevalence of anemia among pregnant women (%)", "Prevalence of anemia among women of reproductive age (% of women ages 15-49)", "People using at least basic sanitation services, rural (% of rural population)", "People using at least basic drinking water services, urban (% of urban population)", "People using at least basic sanitation services, urban (% of urban population)", "Prevalence of undernourishment (% of population)", "Tuberculosis treatment success rate (% of new cases)", "Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)", "Total alcohol consumption per capita, female (liters of pure alcohol, projected estimates, female 15+ years of age)", "Total alcohol consumption per capita, male (liters of pure alcohol, projected estimates, male 15+ years of age)", "Lifetime risk of maternal death (%)", "Lifetime risk of maternal death (1 in: rate varies by country)", "Maternal mortality ratio (modeled estimate, per 100,000 live births)", "Number of maternal deaths", "People using at least basic drinking water services (% of population)", "People practicing open defecation (% of population)", "Mortality rate, adult, female (per 1,000 female adults)", "Mortality rate, adult, male (per 1,000 male adults)", "Tuberculosis case detection rate (%, all forms)", "Out-of-pocket expenditure (% of current health expenditure)", "Out-of-pocket expenditure per capita (current US$)", "Current health expenditure (% of GDP)", "Current health expenditure per capita (current US$)", "Current health expenditure per capita, PPP (current international $)", "Domestic general government health expenditure (% of GDP)", "Domestic general government health expenditure (% of current health expenditure)", "Domestic general government health expenditure per capita (current US$)", "Domestic general government health expenditure per capita, PPP (current international $)", "Domestic private health expenditure (% of current health expenditure)", "Domestic private health expenditure per capita (current US$)", "Domestic private health expenditure per capita, PPP (current international $)", "Out-of-pocket expenditure per capita, PPP (current international $)", "People using at least basic sanitation services (% of population)", "Adolescent fertility rate (births per 1,000 women ages 15-19)", "Birth rate, crude (per 1,000 people)", "Death rate, crude (per 1,000 people)", "Immunization, DPT (% of children ages 12-23 months)", "Immunization, measles (% of children ages 12-23 months)", "Incidence of tuberculosis (per 100,000 people)", "Mortality rate, infant (per 1,000 live births)", "Mortality rate, infant, female (per 1,000 live births)", "Mortality rate, infant, male (per 1,000 live births)", "Mortality rate, neonatal (per 1,000 live births)", "Mortality rate, under-5 (per 1,000 live births)", "Mortality rate, under-5, female (per 1,000 live births)", "Mortality rate, under-5, male (per 1,000 live births)", "Number of deaths ages 10-14 years", "Number of deaths ages 15-19 years", "Number of deaths ages 20-24 years", "Number of deaths ages 5-9 years", "Number of infant deaths", "Number of neonatal deaths", "Number of under-five deaths", "Population ages 0-14 (% of total population)", "Population ages 0-14, female", "Population ages 0-14, female (% of female population)", "Population ages 0-14, male", "Population ages 0-14, male (% of male population)", "Population ages 0-14, total", "Population ages 10-14, female (% of female population)", "Population ages 10-14, male (% of male population)", "Population ages 15-19, female (% of female population)", "Population ages 15-19, male (% of male population)", "Population ages 15-64 (% of total population)", "Probability of dying among adolescents ages 10-14 years (per 1,000)", "Probability of dying among adolescents ages 15-19 years (per 1,000)", "Probability of dying among children ages 5-9 years (per 1,000)", "Probability of dying among youth ages 20-24 years (per 1,000)", "International tourism, number of arrivals", "Fuel exports (% of merchandise exports)", "Insurance and financial services (% of commercial service exports)", "Ores and metals imports (% of merchandise imports)", "Fuel imports (% of merchandise imports)", "International tourism, expenditures for travel items (current US$)", "International tourism, receipts for travel items (current US$)", "Transport services (% of commercial service exports)", "Insurance and financial services (% of commercial service imports)", "Travel services (% of commercial service exports)", "Transport services (% of commercial service imports)", "Travel services (% of commercial service imports)", "Computer, communications and other services (% of commercial service exports)", "Computer, communications and other services (% of commercial service imports)", "Export unit value index (2015 = 100)", "Import unit value index (2015 = 100)", "Refugee population by country or territory of asylum", "Share of youth not in education, employment or training, female (% of female youth population) (modeled ILO estimate)", "Share of youth not in education, employment or training, male (% of male youth population) (modeled ILO estimate)", "Share of youth not in education, employment or training, total (% of youth population) (modeled ILO estimate)", "Refugee population by country or territory of origin", "GDP per person employed (constant 2021 PPP $)", "Employment to population ratio, 15+, female (%) (modeled ILO estimate)", "Employment to population ratio, 15+, male (%) (modeled ILO estimate)", "Employment to population ratio, 15+, total (%) (modeled ILO estimate)", "Employment to population ratio, ages 15-24, female (%) (modeled ILO estimate)", "Employment to population ratio, ages 15-24, male (%) (modeled ILO estimate)", "Employment to population ratio, ages 15-24, total (%) (modeled ILO estimate)", "Labor force participation rate for ages 15-24, female (%) (modeled ILO estimate)", "Labor force participation rate for ages 15-24, male (%) (modeled ILO estimate)", "Labor force participation rate for ages 15-24, total (%) (modeled ILO estimate)", "Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)", "Labor force participation rate, female (% of female population ages 15-64) (modeled ILO estimate)", "Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate)", "Labor force participation rate, male (% of male population ages 15-64) (modeled ILO estimate)", "Labor force participation rate, total (% of total population ages 15+) (modeled ILO estimate)", "Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)", "Labor force, female (% of total labor force)", "Labor force, total", "Net migration", "Ratio of female to male labor force participation rate (%) (modeled ILO estimate)", "Self-employed, female (% of female employment) (modeled ILO estimate)", "Self-employed, male (% of male employment) (modeled ILO estimate)", "Self-employed, total (% of total employment) (modeled ILO estimate)", "Unemployment, female (% of female labor force) (modeled ILO estimate)", "Unemployment, male (% of male labor force) (modeled ILO estimate)", "Unemployment, total (% of total labor force) (modeled ILO estimate)", "Unemployment, youth female (% of female labor force ages 15-24) (modeled ILO estimate)", "Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)", "Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)", "Vulnerable employment, female (% of female employment) (modeled ILO estimate)", "Vulnerable employment, male (% of male employment) (modeled ILO estimate)", "Vulnerable employment, total (% of total employment) (modeled ILO estimate)", "Wage and salaried workers, female (% of female employment) (modeled ILO estimate)", "Wage and salaried workers, male (% of male employment) (modeled ILO estimate)", "Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"

    
]

# Filter your DataFrame to include only these columns
df = dataset_df[columns_to_keep]

# (Optional) Print the filtered DataFrame's columns to verify
df

dataset_missing_report = create_missing_report_by_column_with_source(df, indicator_to_source)

save_csv(dataset_missing_report, '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/Dataset_Missing_Report.csv')

save_csv(df, '/Users/josh/Desktop/Macbook Working Files/Git Repos/culminating-project-group-1/Week 5/Datasets and Research Questions/JC/Group1DatasetFinal.csv')